In [37]:
from textblob import TextBlob
import emoji
import tweepy
import csv
import string
import nltk
import re
import datetime, time
import pickle
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd


In [17]:
CONSUMER_KEY = "gascQ6nxzGIPHAJKVjIVVf3QE"
CONSUMER_SECRET = "4ixOxsZ4rS6yWhlotLNVib465CdmavTUbIvVUCRHlajgdNlfD0"
ACCESS_TOKEN = "858881393727877120-EqrBy6vqwxazAedJw92aTJnRmpnqUr6"
ACCESS_TOKEN_SECRET = "WHYhY1Xd17YUrzOdmx3LUwqzQyz1XOVa5HFPx5ASSglRa"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [36]:
#usernames = ["StockTwits", "YahooFinance", "BobPisani"] 
#usernames = ["Benzinga","jimcramer","stockwits"] 
usernames=["BobPisani"]
#Benzinga
        
def give_emoji_free_text(text):
    allchars = [str for str in text.decode('utf-8')]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
    return clean_text

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with   this method

    #authorize twitter, initialize tweepy
    #auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    #auth.set_access_token(access_token, access_token_secret)
    #api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    print("screen name",screen_name)
    #make initial request for most recent tweets (200 is the maximum   allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1
    
    count=0
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        count+=1
    #new_tweets = api.user_timeline(screen_name = screen_name,count=30000)

    #save most recent tweets
    #alltweets.extend(new_tweets)

    #save the id of the oldest tweet less one
   
    print("...%s tweets downloaded so far" % (len(alltweets)))
        
    stop_words = set(stopwords.words('english'))
    stop_words.remove('up') 
    stop_words.remove('down')
    stop_words.remove('out')
    stop_words.remove('off')
    stop_words.remove('under')
    stop_words.remove('over')
    
    #transform the tweepy tweets into a 2D array that will populate the csv 
    #outtweets = [[tweet.id_str, tweet.created_at, 
                  #give_emoji_free_text(str.split(str(tweet.text.encode("utf-8")),'https')[0])] for tweet in alltweets]
    #outtweets = [[tweet.id_str, tweet.created_at, 
                  #str.split(give_emoji_free_text(tweet.text.encode("utf-8")),'https')[0]] for tweet in alltweets]
    outtweets = [[tweet.created_at,
                str.split(give_emoji_free_text(tweet.text.encode("utf-8")),'https')[0]] for tweet in alltweets]
    filteredlist=[]
    for notweet in range(len(outtweets)):
        word_tokens = word_tokenize(str(outtweets[notweet][1]))
        filtered_sentence = [w for w in word_tokens if not w.upper() in [item.upper() for item in stop_words]]
        #strf='"' +' '.join(filtered_sentence)+ '"'
        strf=' '.join(filtered_sentence)
        filteredlist.append(strf)
        
    createddates=[]
    tweets=[]
    
    for flist in range(len(filteredlist)):
        outtweets[flist][1]=filteredlist[flist]
    totaltweets={}
    for flis in range(len(outtweets)):
        createddates.append(outtweets[flis][0])
        tweets.append(outtweets[flis][1])
    totaltweets["created_date"]=createddates
    totaltweets["tweets"]=tweets
    df = pd.DataFrame(totaltweets)
    
    calculatePolarities(df,screen_name)
    #write the csv  
    '''with open('%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["created_at","text"])
        writer.writerows(outtweets)'''

    #pass
def calculatePolarities(df,screen_name):
    dfcp=df.copy()
    
    
    tweetset=df["tweets"]
    polarit=[]
    for tweet in tweetset:
        analysis=TextBlob(tweet)
        polarityval=analysis.sentiment.polarity
        #print("plo%d",polarityval)
        
        if(polarityval==0):
            polarit.append("Neutral")
        elif(polarityval<0):
            
            polarit.append("Negative")
        elif(polarityval>0):
            
            polarit.append("Positive")
    
    dfcp["sentiment"]=polarit
    dfcp.to_csv('{0}_tweets.csv'.format(screen_name))
    
    
if __name__ == '__main__':
    #pass in the username of the account you want to download
    for x in usernames:
        get_all_tweets(x)

screen name BobPisani
getting tweets before 1004769626633277439
getting tweets before 917359531243593727
getting tweets before 856655540021276671
getting tweets before 818942015324831744
getting tweets before 768050432287526911
getting tweets before 715922795235045376
getting tweets before 672119967286865919
getting tweets before 626459796066172927
getting tweets before 585813623391457279
getting tweets before 534361069519904768
getting tweets before 520608291919892479
getting tweets before 504985749880442879
getting tweets before 487231001178165247
getting tweets before 476731269859278847
getting tweets before 460818477113417727
getting tweets before 445935938037092351
getting tweets before 444165022013730815
...3215 tweets downloaded so far


In [ ]:
''' oldest = alltweets[-1].id - 1
    count=0
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=3000,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        count+=1'''